# PLANMAP Readme to JSON metadata file

In this note we will translate a Planmap package Readme.md (Markdown) to a JSON file and validate it against the GMAP package jsonschema, then save it.

In [1]:
PUB_DIR = 'data/publish_data/archive/'
ZIP_DIR = 'data/publish_data/zip/'

In [2]:
readme_file = f'{PUB_DIR}/mars/PM-MAR-MS-Arsinoes/README.md'

with open(readme_file, 'r') as fp:
    readme = fp.read()
    

In [3]:
print(readme)

---
<img src="./document/PM-MAR-MS-Arsinoes.browse.png" width="800"/>

| Field                                                        | Description                                                  |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| Map name (PM_ID)                                             | PM-MAR-MS-Arsinoes                                           |
| Map version                                                  | 03                                        |
| Target body                                                  | Mars                                                         |
| Title of map                                                 | Geological Map of Arsinoes and Pyrrhae Chaos, Mars           |
| Bounding box - Min Lat                                       | -12                                                          |
| Bounding box - Max Lat                            

In [4]:
# Import Planmap package
#
import api.planmap

from importlib import reload
reload(api.planmap)

table = api.planmap.markdown2json(readme)

api.planmap.validate(table)

In [5]:
def validate_readme(readme_file:str) -> dict:
    """
    Return 'readme' as a json if validatation is succesfull
    
    Validation JSONSchema: `api.planmap.README_SCHEMA`
    """
    import api.planmap
    
    with open(readme_file, 'r') as fp:
        readme = fp.read()

    table = api.planmap.markdown2json(readme)

    api.planmap.validate(table)
    
    return table


In [6]:
import itertools

In [7]:
import os
from glob import glob

# Look for Planmap packages' Readme in every body, Mars, Mercury, Moon
#

def resolve_paths(path_template="{basedir}/{bodies}/PM-*", **kwargs):
    # Ex. kwargs: basedir=PUB_DIR, bodies=['mars','mercury','moon']
    #
    for k,v in kwargs.items():
        kwargs[k] = [v] if isinstance(v, str) else v
        
    product = [ v for v in itertools.product(*kwargs.values()) ]
    path_values = [ { k:v for k,v in zip(kwargs.keys(),prod) } for prod in product ]

    paths = []
    for kwargs in path_values:
        # Resolve path template for current 'kwargs'
        #
        dirs_regex = path_template.format(**kwargs)
        try:
            pkg_dirs = glob(f"{dirs_regex}")
        except:
            return None
        
        paths.extend(pkg_dirs)
            
    return paths


In [8]:
pkg_dirs = resolve_paths(path_template="{basedir}/{bodies}/PM-*", basedir=PUB_DIR, bodies=['mars','mercury','moon'])

table_all = {}

for pkg_dir in pkg_dirs:
    pkg_name = os.path.basename(pkg_dir)
    for filepath in glob(f"{pkg_dir}/*"):
        if os.path.basename(filepath).lower() == 'readme.md':
            print('--', filepath)
            table = validate_readme(filepath)
            assert pkg_name not in table_all
            table_all.update({ pkg_name: table })


-- data/publish_data/archive//mars/PM-MAR-MS-Crommelin/README.md
-- data/publish_data/archive//mars/PM-MAR-C-Crommelin/README.md
-- data/publish_data/archive//mars/PM-MAR-D-Oxia_Planum/README.md
-- data/publish_data/archive//mars/PM-MAR-D-Crommelin/README.md
-- data/publish_data/archive//mars/PM-MAR-MS-Arsinoes/README.md
-- data/publish_data/archive//mars/PM-MAR-SI-Crommelin/README.MD
-- data/publish_data/archive//mars/PM-MAR-D-Gale_Murray/readme.md
-- data/publish_data/archive//mars/PM-MAR-C-Arsinoes/README.md
-- data/publish_data/archive//mars/PM-MAR-D-Jezero/README.md
-- data/publish_data/archive//mars/PM-MAR-D-Gale_Kimberley/README.md
-- data/publish_data/archive//mercury/PM-MER-C-H05_SU/README.md
-- data/publish_data/archive//mercury/PM-MER-GS-Rachmaninoff_5cc/README.md
-- data/publish_data/archive//mercury/PM-MER-MS-Beethoven/README.md
-- data/publish_data/archive//mercury/PM-MER-MS-H14_5cc/README.MD
-- data/publish_data/archive//mercury/PM-MER-MS-H10_5cc/README.MD
-- data/publis

In [9]:
import pandas as pd

pd.set_option('display.max_columns',100)

pd.__version__

'1.4.3'

In [10]:
pd.DataFrame(table_all).T

,Map name (PM_ID),Target body,Title of map,Bounding box - Min Lat,Bounding box - Max Lat,Bounding box - Min Lon (0-360),Bounding box - Max Lon (0-360),Author(s),Type,Output scale,Original Coordinate Reference System,Data used,Standards adhered to,DOI,Aims (one sentence),Short description,Related products (cross link to other Planmap products),Units Definition,Stratigraphic info (e.g. production function used),"Other comments (reviewer comments, notes on post-processing)",Heritage used,Link to other repositories,Acknowledgements beyond Planmap,Map version,"Standards adhered to open formats for 3D mesh, materials and textures (obj, .mtl, .png)",Polygon number,Number of attached textures,Modelling method,Aims,Related products,Units Definition (polygon styling),Stratigraphic info,Other comments,Standards
PM-MAR-MS-Crommelin,PM-MAR-MS-Crommelin,Mars,"Geological Map of the Crommelin Crater, Mars",3.96993,6.22761,348.8477,350.98821,"D. Pesce, R. Pozzobon, M. Massironi",Preliminary,1:160.000,Projected Coordinate System: Equirectangular P...,MOLA Elevation Model MEGDR (463 m/pixel)HRSC s...,"Mapping scale: Tobler (1987), Planmap mapping ...",,Morpho-stratigraphic mapping,This 1:160.000 scale map by Pesce et al. cover...,,"Aeolian deposits, AD- Very dark toned, AD1, 78...",,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM-MAR-C-Crommelin,PM-MAR-C-Crommelin,Mars,Spectral Map to highlight color or composition...,3.96993,6.22761,348.8477,350.98821,Carli C.,Preliminary,Publication scale,CRS ???TBC,"CRISM images TRR3 (FRT, FRS, HRS and HRL)",TBC,,spectroscopical information related to spectra...,,,"Units names, Codes, RGB colors",,"Upper Left (10008396.577, 369139.888) ( 11d ...",Viviano-Beck C.E. et al. 2014,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM-MAR-D-Oxia_Planum,PM-MAR-D-Oxia_Planum,Mars,SIG and 3D data of the Oxia Planum area,17.34694,18.94444,25.25278,25.68028,"G. Caravaca, S. Le Mouélic, N. Mangold",Final,Dynamic,Projected Coordinate System: Equirectangular_M...,DEM from: CTX DTM (18 m)\tImagery (visible): C...,NaN,TBD,GIS data of the Oxia Planum future landing sit...,Orbital data released with D5.4 related to the...,Deliverable 5.4 (Virtual Reality application)\...,,,,Morpho-textural map released on ESA's website,,We thank the MarsSI team and Cathy Quantin-Nat...,1.0,,N/A (generated by application engine),N/A (generated by application engine),Orbital model: DEM Extrusion using application...,NaN,NaN,NaN,NaN,NaN,NaN
PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,Mars,3D subsurface geomodel of western Crommelin cr...,3.96993,6.22761,348.8477,350.98821,"R. Pozzobon, D. Pesce, M. Massironi",Preliminary,1:160.000,Projected Coordinate System: Equirectangular; ...,Merged DEM from: MOLA Elevation Model MEGDR (4...,NaN,,3D geomodel of Crommelin Crater's inner layere...,This is the 3D geomodel released within D6.1 a...,Deliverable D2.2 (morphostratigraphic maps); D...,,,,,Crommelin crater with image mosaic and with th...,,1.0,,Mesh in data repository (topography+geomodel):...,2 textures (MS map and image mosaic),Topography: Stereo DEM extrusion; Geomodel: ex...,NaN,NaN,NaN,NaN,NaN,NaN
PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,Mars,"Geological Map of Arsinoes and Pyrrhae Chaos, ...",-12.0,-5.8,329.7,334.5,"E. Luzzi, A.P. Rossi",Preliminary,1:3.000.000,Projected Coordinate System: Equirectangular P...,MOLA Elevation Model MEGDR (463 m/pixel)CTX mo...,Planmap mapping standards document,,Morpho-stratigraphic mapping,This map shows the contacts between the disrup...,,"Post-collapse craters, PCC, 51-160-44Cap Unit,...",,,Glotch and Christensen 2005,,,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM-MAR-SI-Crommelin,PM-MAR-SI-Crommelin,Mars,Geo-stratigraphic map of NW sector of the Crom...,18.1945,7.560542,155.894958,357.867708,"G. Melchiori, M. Massironi, R. Pozzobon",Released,1:100000,Mars Equirectangular,"CTX, CRISM",Planmap mapping standards document,,Geo-stratigraphic map of the inner layered dep...,,"PM-MAR-MS-Crommelin_01, PM-MAR-D-Crommelin_01",<

In [11]:
# Let's "fix" a couple of details:
# * Column names:
#   - 'Aims (one sentence)' -> 'Aims'
#   - 'Related products (cross link to other Planmap products)' -> 'Related products'
#   - 'Units Definition (polygon styling)' -> 'Units definition'
#   - 'Other comments (reviewer comments, notes on post-processing)' -> 'Other comments'
#   - 'Stratigraphic info (e.g. production function used)' -> 'Stratigraphic info'
#   - 'Standards adhered to' -> 'Standards'

mappings = {
    'Aims (one sentence)': 'Aims',
    'Related products (cross link to other Planmap products)': 'Related products',
    'Units Definition (polygon styling)': 'Units Definition',
    'Other comments (reviewer comments, notes on post-processing)': 'Other comments',
    'Stratigraphic info (e.g. production function used)': 'Stratigraphic info',
    'Standards adhered to': 'Standards'
}

def map_columns(table_all, mappings):
    """
    Return new dict/json with mappings applied
    """
    new_all = {}
    for name, table in table_all.items():
        new_table = {}
        for field, value in table.items():
            if isinstance(value, str):
                if any([ value.lower() == na for na in ('na','nan','n/a') ]) or value.strip() == '':
                    value = None
            if isinstance(value, float) and value != value:
                value = None
                
            if field in mappings:
                field = mappings[field]
                
            new_table[field] = value
            
        new_all[name] = new_table
    
    return new_all


In [12]:
table_tmp = map_columns(table_all, mappings)

df_tmp = pd.DataFrame(table_tmp.values(), index=table_tmp.keys())
# df_tmp

> We now check the column-values to confirm the mapping worked fine. And then, remove the old columns

In [13]:
# Check if "Map name" and the index (from directory name) have the same value
#
from IPython.display import display

pmid_column = 'Map name (PM_ID)'

if sum(names_checked := df_tmp.index != df_tmp[pmid_column]):
    print("Packages with directory (index) & 'PM_ID' not maching:\n")
    display(df_tmp.loc[names_checked][pmid_column])
else:
    print("PM_ID & Index check equal. 'PM_ID' can subtitute index.")
    # df_tmp.set_index(pmid_column, drop=True, inplace=True)

PM_ID & Index check equal. 'PM_ID' can subtitute index.


In [14]:
# Parse authors column
#

authors_column = 'Author(s)'

authors = df_tmp[authors_column].apply(lambda names: 
                                       [n.strip() for n in names.replace(' and ','').replace(',',';').split(';')] 
                                       if names else None)
df_tmp[authors_column] = authors

In [15]:
# df_tmp[authors_column] != None
# df_tmp = df_tmp.loc[df_tmp[authors_column] != None]
df_tmp

,Map name (PM_ID),Target body,Title of map,Bounding box - Min Lat,Bounding box - Max Lat,Bounding box - Min Lon (0-360),Bounding box - Max Lon (0-360),Author(s),Type,Output scale,Original Coordinate Reference System,Data used,Standards,DOI,Aims,Short description,Related products,Units Definition,Stratigraphic info,Other comments,Heritage used,Link to other repositories,Acknowledgements beyond Planmap,Map version,"Standards adhered to open formats for 3D mesh, materials and textures (obj, .mtl, .png)",Polygon number,Number of attached textures,Modelling method
PM-MAR-MS-Crommelin,PM-MAR-MS-Crommelin,Mars,"Geological Map of the Crommelin Crater, Mars",3.969930,6.227610,348.847700,350.988210,"[D. Pesce, R. Pozzobon, M. Massironi]",Preliminary,1:160.000,Projected Coordinate System: Equirectangular P...,MOLA Elevation Model MEGDR (463 m/pixel)HRSC s...,"Mapping scale: Tobler (1987), Planmap mapping ...",None,Morpho-stratigraphic mapping,This 1:160.000 scale map by Pesce et al. cover...,None,"Aeolian deposits, AD- Very dark toned, AD1, 78...",None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
PM-MAR-C-Crommelin,PM-MAR-C-Crommelin,Mars,Spectral Map to highlight color or composition...,3.969930,6.227610,348.847700,350.988210,[Carli C.],Preliminary,Publication scale,CRS ???TBC,"CRISM images TRR3 (FRT, FRS, HRS and HRL)",TBC,None,spectroscopical information related to spectra...,None,None,"Units names, Codes, RGB colors",None,"Upper Left (10008396.577, 369139.888) ( 11d ...",Viviano-Beck C.E. et al. 2014,None,None,NaN,NaN,NaN,NaN,NaN
PM-MAR-D-Oxia_Planum,PM-MAR-D-Oxia_Planum,Mars,SIG and 3D data of the Oxia Planum area,17.346940,18.944440,25.252780,25.680280,"[G. Caravaca, S. Le Mouélic, N. Mangold]",Final,Dynamic,Projected Coordinate System: Equirectangular_M...,DEM from: CTX DTM (18 m)\tImagery (visible): C...,NaN,TBD,GIS data of the Oxia Planum future landing sit...,Orbital data released with D5.4 related to the...,Deliverable 5.4 (Virtual Reality application)\...,None,None,None,Morpho-textural map released on ESA's website,None,We thank the MarsSI team and Cathy Quantin-Nat...,1.0,None,N/A (generated by application engine),N/A (generated by application engine),Orbital model: DEM Extrusion using application...
PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,Mars,3D subsurface geomodel of western Crommelin cr...,3.969930,6.227610,348.847700,350.988210,"[R. Pozzobon, D. Pesce, M. Massironi]",Preliminary,1:160.000,Projected Coordinate System: Equirectangular; ...,Merged DEM from: MOLA Elevation Model MEGDR (4...,NaN,None,3D geomodel of Crommelin Crater's inner layere...,This is the 3D geomodel released within D6.1 a...,Deliverable D2.2 (morphostratigraphic maps); D...,None,None,None,None,Crommelin crater with image mosaic and with th...,None,1.0,None,Mesh in data repository (topography+geomodel):...,2 textures (MS map and image mosaic),Topography: Stereo DEM extrusion; Geomodel: ex...
PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,Mars,"Geological Map of Arsinoes and Pyrrhae Chaos, ...",-12.000000,-5.800000,329.700000,334.500000,"[E. Luzzi, A.P. Rossi]",Preliminary,1:3.000.000,Projected Coordinate System: Equirectangular P...,MOLA Elevation Model MEGDR (463 m/pixel)CTX mo...,Planmap mapping standards document,None,Morpho-stratigraphic mapping,This map shows the contacts between the disrup...,None,"Post-collapse craters, PCC, 51-160-44Cap Unit,...",None,None,Glotch and Christensen 2005,None,None,3.0,NaN,NaN,NaN,NaN
PM-MAR-SI-Crommelin,PM-MAR-SI-Crommelin,Mars,Geo-stratigraphic map of NW sector of the Crom...,18.194500,7.560542,155.894958,357.867708,"[G. Melchiori, M. Massironi, R. Pozzobon]",Released,1:100000,Mars Equirectangular,"CTX, CRISM",Planmap mapping standards document,None,Geo-stratigraphic map of the inner layered dep...,None,"PM-MAR-MS-Crommelin_01, PM-MAR-D-Crommelin_01",<ul>Aeolian Deposits: Sand and Silty deposits ...,None,None,PM-MAR-MS-Crommelin,None,None,1.0,NaN,NaN,NaN,NaN
PM-MAR-D-Gale_Murray,PM-MAR-D-Gale_Murray,Mars,Digital Outcrop Model (DO

In [16]:
import json

table_tmp = json.loads(df_tmp.to_json(orient='records'))

In [17]:
table_tmp = { obj[pmid_column]:obj for obj in table_tmp }

In [18]:
table_clean = table_tmp
del table_tmp

## Save JSON files in their respective packages

Now we go back to where we began -- walking through the packages -- to write the resultant json tables. 
We will call them `meta.json`.

In [19]:
import json
from pathlib import Path

metafile = 'meta.json'

for pkg_dir in pkg_dirs:
    pkg_name = os.path.basename(pkg_dir)
    assert pkg_name in table_clean, f"{list(table_clean.keys())}"

    filepath = Path(pkg_dir) / metafile
    # if filepath.exists():
    #     print(f"Overwritting {filepath} ..")
    
    with open(filepath, 'w') as fp:
        _tab = table_clean[pkg_name]
        json.dump(_tab, fp)
        print(f"Package '{pkg_name}' metadata (README.md) written to '{filepath}'")
        
    # for filepath in glob(f"{pkg_dir}/*"):
    #     if os.path.basename(filepath).lower() == 'readme.md':
    #         print('--', filepath)
    #         table = validate_readme(filepath)
    #         assert pkg_name not in table_all
    #         table_all.update({ pkg_name: table })


Package 'PM-MAR-MS-Crommelin' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-MS-Crommelin/meta.json'
Package 'PM-MAR-C-Crommelin' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-C-Crommelin/meta.json'
Package 'PM-MAR-D-Oxia_Planum' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-D-Oxia_Planum/meta.json'
Package 'PM-MAR-D-Crommelin' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-D-Crommelin/meta.json'
Package 'PM-MAR-MS-Arsinoes' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-MS-Arsinoes/meta.json'
Package 'PM-MAR-SI-Crommelin' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-SI-Crommelin/meta.json'
Package 'PM-MAR-D-Gale_Murray' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-D-Gale_Murray/meta.json'
Package 'PM-MAR-C-Arsinoes' metadata (README.md) written to 'data/publish_data/archive/mars/PM-MAR-C-Arsinoes/meta.json'
Packag

In [20]:
# from importlib import reload
# reload(api.planmap)

# # payload = api.planmap.parse_readme_url('https://data.planmap.eu/pub/mars/PM-MAR-MS-Arsinoes/README.md')
# payload = api.planmap.parse_package(f'{PUB_DIR}/mars/PM-MAR-MS-Arsinoes')

# payload.add_files(f"{ZIP_DIR}/mars/PM-MAR-MS-Arsinoes.zip")

# payload.asdict()

# payload.create_files_payload()

# import api.invenio

# reload(api.invenio)

# # Invenio server
# #
# hostname = '10.72.2.28:5000'
# token = 'MJg5k2AYs5hr1YSXVKH4IifydN70WjUwcjbgxiKlxUaX2ETgB0nma4FiKZfx'

# client = api.invenio.InvenioClient(hostname, token)

# # Create a draft
# #
# resp = client.create_draft(payload)

# # print(resp)

# print(resp['id'])

# # Open record's page
# #
# import webbrowser

# webbrowser.open(resp['links']['self_html'])

# resp

# client.publish_draft(resp['id'])

